In [54]:
import matplotlib.pyplot as plt
import json
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint 
from census import Census

In [55]:
from config import (G_key, Census_key)
gmaps.configure(api_key=G_key)

Store Part I results into DataFrame

Load the csv exported in Part I to a DataFrame

In [56]:
Vacation_df = pd.read_csv('Clean_city_data.csv')
Vacation_df.dropna(inplace = True)
Vacation_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Provideniya,64.38,-173.30,8.00,93,90,4.00,RU,1594762686
1,Ribeira Grande,38.52,-28.70,22.00,78,20,7.20,PT,1594762686
2,Jamestown,42.10,-79.24,23.33,56,20,2.60,US,1594762622
3,Beihai,21.48,109.10,29.62,75,57,3.32,CN,1594762686
4,Thompson,55.74,-97.86,17.00,88,90,4.10,CA,1594762648


Humidity Heatmap

In [57]:
locations = Vacation_df[["Lat", "Lng"]]
Humidity = Vacation_df["Humidity"]

In [59]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

Create new DataFrame fitting weather criteria

In [60]:
Prefered_Max_Temp = (Vacation_df["Max Temp"] < 30)&(Vacation_df["Max Temp"] > 20)
Prefered_Cloudiness = Vacation_df["Cloudiness"] < 5
Prefered_Wind_Speed = Vacation_df["Wind Speed"] < 5
Prefered_Weather = Vacation_df[Prefered_Max_Temp&Prefered_Cloudiness&Prefered_Wind_Speed]
Prefered_Weather = Prefered_Weather.dropna()
Prefered_Weather.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
14,Caravelas,-17.71,-39.25,22.30,85,3,2.83,BR,1594762687
23,Viranşehir,37.24,39.76,27.53,24,0,1.99,TR,1594762688
27,Dawlatabad,36.41,64.91,27.01,17,0,2.45,AF,1594762688
57,Albany,42.60,-73.97,25.56,68,3,3.01,US,1594762691
58,Ponta do Sol,32.67,-17.10,21.67,61,0,2.65,PT,1594762691


Hotel Map

In [61]:
hotel_df = Prefered_Weather
hotel_df["Hotel Name"] = ""

In [94]:
params = {"radius": 5000,"type": "lodging","keyword":"hotel","key": G_key}

In [96]:
params["location"] = f"{Lat},{Lng}"
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
response = requests.get(base_url, params=params).json()

In [97]:
for index, row in hotel_df.iterrows():
    Lat = row["Lat"]
    Lng = row["Lng"]
    params["location"] = f"{Lat},{Lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params).json()
   
    try:
        print(f"{response['results'][0]['name']}.")
        
        hotel_df.loc[index, "Hotel Name"] = response['results'][0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Mar Doce de Abrolhos.
------------
Tella Hotel.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Estalagem Da Ponta Do Sol.
------------
Hotel La Quiete.
------------
Hotel Castrum Villae.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Elysium Hotel.
------------
Mini-hotel Meduza.
------------
Desert Gat.
------------
Missing field/result... skipping.
------------
7 Days Premium.
------------
SureStay Plus Hotel by Best Western Moses Lake.
------------
Sangho Privilege Tataouine.
------------
Hôtel Opémiska.
------------
Safari Hotel.
------------
Missing field/result... skipping.
------------
Rockyview Hotel.
------------
Ali Khan Hotel.
------------
City Convenient Hotel.
------------
Missing field/result... skipping.
------------
Marks Hotel.
------------
Hotel Medena.
------------
Grand Hotel Punta Molino.
------------
ilman Yousafzai.
------------
SpringHill Suite

In [98]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
14,Caravelas,-17.71,-39.25,22.30,85,3,2.83,BR,1594762687,Mar Doce de Abrolhos
23,Viranşehir,37.24,39.76,27.53,24,0,1.99,TR,1594762688,Tella Hotel
27,Dawlatabad,36.41,64.91,27.01,17,0,2.45,AF,1594762688,Dawlat Abad
57,Albany,42.60,-73.97,25.56,68,3,3.01,US,1594762691,Voorheesville
58,Ponta do Sol,32.67,-17.10,21.67,61,0,2.65,PT,1594762691,Estalagem Da Ponta Do Sol
102,Lauria,40.05,15.84,22.22,60,2,1.27,IT,1594762696,Hotel La Quiete
106,Vila,42.03,-8.16,21.67,60,3,4.60,PT,1594762696,Hotel Castrum Villae
117,Néa Málgara,40.61,22.68,22.22,87,0,0.89,GR,1594762697,Chalastra
125,Berdigestyakh,62.10,126.70,20.80,51,0,1.34,RU,1594762698,Berdigestyakh
193,Paphos,34.77,32.42,25.00,94,0,3.10,CY,1594762705,Elysium Hotel


In [99]:
coordinates = hotel_df[["Lat","Lng"]].astype(float)
hotel_df = hotel_df.dropna()
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

In [100]:
markers = gmaps.marker_layer(coordinates)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

In [101]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

In [102]:
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [103]:
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

In [104]:
hotel_layer = gmaps.symbol_layer(
    coordinates, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
)


fig = gmaps.figure()
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [105]:
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))